Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [8]:
from itertools import product
from random import random, randint, shuffle, seed
import numpy as np
from scipy import sparse
from functools import reduce

In [9]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

In [10]:
def fitness(state):
    cost = state.sum(axis=0)
    global fitness_counter
    fitness_counter = fitness_counter + 1
    valid = np.all(
        reduce(
            np.logical_or,
            [x[[i], :].toarray()[0] for i, t in enumerate(state) if t],
            np.array([False for _ in range(problem_dim)]),
        )
    )
    return valid, cost

In [ ]:
# function to generate a random initial solution (random selection of sets)
def initialize_solution(num_sets):
    selected_sets = [str(i) for i in range(num_sets) if random() < 0.5]
    return ','.join(selected_sets)

In [ ]:
# evolutionary algorithm single state --> Genetic Algorithm
def genetic_algorithm(problem, max_generations, population_size):
    best_solution = None
    best_fitness = float('inf')
    population = []

    for _ in range(population_size):
        population.append(initialize_solution(num_sets))

    for generation in range(max_generations):
        new_population = []

        for _ in range(population_size):
            # Select two parents randomly from the population
            parent1, parent2 = np.random.choice(population, size=2, replace=False)

            # Perform crossover and mutation to create a new child solution
            crossover_point = randint(1, num_sets - 1)
            child = parent1[:crossover_point] + parent2[crossover_point:]

            if random() < 0.1:
                mutation_point = randint(0, num_sets - 1)
                child = child[:mutation_point] + str(1 - int(child[mutation_point])) + child[mutation_point + 1:]

            # Evaluate the fitness of the child
            valid, cost = fitness(child, problem)

            # Add the child to the new population
            new_population.append(child)

            # Update the best solution if the child is better
            if valid and cost < best_fitness:
                best_solution = child
                best_fitness = cost

        # Replace the old population with the new population
        population = new_population

        # Output the best solution at the end of each generation
        print(f"Generation {generation}: Best solution = {best_solution}, Cost = {best_fitness}")

    return best_solution, best_fitness

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [11]:
num_points = 100
num_sets = num_points
density = 0.3
x = make_set_covering_problem(num_points, num_sets, density)
print("Element at row=42 and column=42:", x[42, 42])

Element at row=42 and column=42: False
